In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time
from utils import lower_and_remove_diacritics

In [31]:
#artists = ["Mišo Kovač", "Prljavo Kazalište", "Crvena Jabuka", "Gibonni", "Zlatko Pejaković", "Leteći Odred", "Severina", "Doris Dragović", "Thompson", "Danijela Martinović", "Petar Grašo", "Oliver Dragojević", "Colonia", "Novi Fosili", "Parni Valjak", "Plavi Orkestar", "Zabranjeno Pušenje"]
#artists_diskografija = ["Mate Mišo Kovač", "Prljavo Kazalište", "Crvena Jabuka", "Gibonni", "Zlatko Pejaković", "Leteći Odred", "Severina", "Doris Dragović", "Thompson", "Danijela Martinović", "Petar Grašo", "Oliver", "Colonia", "Novi Fosili", "Parni Valjak", "Plavi Orkestar", "Zabranjeno pušenje"]


In [2]:
artists = ["Hladno Pivo", "Mladen Grdović", "Mate Bulić", "Vinko Coce", "Jasmin Stavros", "Boris Novković"]
artists_diskografija = ["Hladno Pivo", "Mladen Grdović", "Mate Bulić", "Vinko Coce", "Jasmin Stavros", "Boris Novković"]

In [3]:
def search_google(search_query):
     #search
    driver.find_element(By.NAME, 'q').clear()
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

In [4]:
#tekstovinet link
ser = Service("chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://google.com")
time.sleep(0.5)
try: driver.find_element(By.XPATH, "//*[contains(text(), 'Aceito')]").click()
except: driver.find_element(By.XPATH, "//*[contains(text(), 'I agree')]").click()
time.sleep(0.5)
try: driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()
except: pass
time.sleep(0.5)


urls_tekstovinet = []
artist_ids_discogs = []
urls_cuspajz = []



for artist in artists:

    search_google(f"{artist} tekstovi.net")
    time.sleep(2)
    try: 
        driver.find_element(By.PARTIAL_LINK_TEXT, 'tekstovi.net').send_keys(Keys.CONTROL + Keys.RETURN)
        time.sleep(0.5)
        window_handles = driver.window_handles
        driver.switch_to.window(window_name=window_handles[1])    
        time.sleep(1)

        artist_tekstovinet = driver.find_element(By.CLASS_NAME, 'lyricCapt').text

        if lower_and_remove_diacritics(artist_tekstovinet) == lower_and_remove_diacritics(artist):
            url_tekstovinet = driver.current_url
        else: url_tekstovinet = np.nan
        driver.close()
        driver.switch_to.window(window_name=window_handles[0])
    except: url_tekstovinet = np.nan


    #cuspajz
    search_google(f"{artist} cuspajz")
    time.sleep(2)
    try:
        driver.find_element(By.PARTIAL_LINK_TEXT, 'cuspajz.com').send_keys(Keys.CONTROL + Keys.RETURN)
        time.sleep(0.5)
        window_handles = driver.window_handles
        driver.switch_to.window(window_name=window_handles[1])    
        time.sleep(1)

        artist_cuspajz = driver.find_element(By.CLASS_NAME, 'songList').text

        if lower_and_remove_diacritics(artist_cuspajz) == lower_and_remove_diacritics(artist):
            url_cuspajz = driver.current_url
        else: url_cuspajz = np.nan
        driver.close()
        driver.switch_to.window(window_name=window_handles[0])
    except: url_cuspajz = np.nan




    #discogs
    search_google(f"{artist} discogs")
    time.sleep(2)
    try:
        driver.find_element(By.PARTIAL_LINK_TEXT, 'discogs.com').send_keys(Keys.CONTROL + Keys.RETURN)
        time.sleep(0.5)
        window_handles = driver.window_handles
        driver.switch_to.window(window_name=window_handles[1])    
        time.sleep(1)

        artist_discogs = driver.find_element(By.CLASS_NAME, 'hide_mobile').text

        if lower_and_remove_diacritics(artist_discogs) == lower_and_remove_diacritics(artist):
            url_discogs = driver.current_url
            artist_id_discogs = int(url_discogs[url_discogs.index("artist/")+7:url_discogs.index("-")])
        else: artist_id_discogs = np.nan
        driver.close()
        driver.switch_to.window(window_name=window_handles[0])
    except: artist_id_discogs = np.nan

    
    urls_tekstovinet.append(url_tekstovinet)
    artist_ids_discogs.append(artist_id_discogs)
    urls_cuspajz.append(url_cuspajz)

driver.quit()


In [5]:
df = pd.DataFrame([artists, artists_diskografija, urls_tekstovinet, urls_cuspajz, artist_ids_discogs]).transpose()
df.columns = ["artist_name", "artist_name_diskografija", "url_tekstovinet", "url_cuspajz", "artist_id_discogs"]

In [6]:
import discogs_client
d = discogs_client.Client('datamilas/0.1', user_token='BDGKHOLMZJxywoowKXvyRhXeqJhkezYxwVjxvBJR')
for a in df.artist_id_discogs:
    print(d.artist(a))


ModuleNotFoundError: No module named 'discogs_client'

In [9]:
df

,artist_name,artist_name_diskografija,url_tekstovinet,url_cuspajz,artist_id_discogs
0,Hladno Pivo,Hladno Pivo,"https://tekstovi.net/2,712,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,895602
1,Mladen Grdović,Mladen Grdović,"https://tekstovi.net/2,777,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,345896
2,Mate Bulić,Mate Bulić,"https://tekstovi.net/2,729,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,2510098
3,Vinko Coce,Vinko Coce,"https://tekstovi.net/2,972,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,995276
4,Jasmin Stavros,Jasmin Stavros,"https://tekstovi.net/2,536,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,1135414
5,Boris Novković,Boris Novković,"https://tekstovi.net/2,328,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,474052


In [8]:

df.loc[2, "artist_id_discogs"] = 2510098


In [10]:
old_df = pd.read_csv("data/info_for_scraping.csv")

In [19]:
new_df = old_df.append(df)
new_df.reset_index(drop=True, inplace=True)

In [20]:
new_df

,artist_name,artist_name_diskografija,url_tekstovinet,url_cuspajz,artist_id_discogs
0,Mišo Kovač,Mate Mišo Kovač,"https://tekstovi.net/2,692,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,469595
1,Prljavo Kazalište,Prljavo Kazalište,"https://tekstovi.net/2,206,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,578760
2,Crvena Jabuka,Crvena Jabuka,"https://tekstovi.net/2,240,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,503226
3,Gibonni,Gibonni,"https://tekstovi.net/2,153,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,637974
4,Zlatko Pejaković,Zlatko Pejaković,"https://tekstovi.net/2,685,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,846393
5,Leteći Odred,Leteći Odred,"https://tekstovi.net/2,480,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,2182231
6,Severina,Severina,"https://tekstovi.net/2,212,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,473824
7,Doris Dragović,Doris Dragović,"https://tekstovi.net/2,518,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,793032
8,Thompson,Thompson,"https://tekstovi.net/2,440,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,470962
9,Danijela Martinović,Danijela Martinović,"https://tekstovi.net/2,253,0.html",https://cuspajz.com/tekstovi-pjesama/izvodjac/...,1162803


In [22]:
new_df.to_csv("info_for_scraping.csv", index=False)